## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-04-19-43-12 +0000,nyt,"Mark Sanchez, Ex-N.F.L. Quarterback, Is Injure...",https://www.nytimes.com/2025/10/04/us/mark-san...
1,2025-10-04-19-43-09 +0000,nypost,"Hundreds attend wake for Isabella Salas, NJ te...",https://nypost.com/2025/10/04/us-news/hundreds...
2,2025-10-04-19-42-23 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/04/world/...
3,2025-10-04-19-40-52 +0000,nyt,Israel and Hamas Say They’ll Work With Trump’s...,https://www.nytimes.com/2025/10/04/world/middl...
4,2025-10-04-19-37-19 +0000,cbc,2 Canadians detained in Israel after forces in...,https://www.cbc.ca/news/politics/gaza-flotilla...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,45
26,gaza,26
27,plan,18
23,hamas,16
22,israel,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
162,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...,150
72,2025-10-04-14-18-43 +0000,nypost,Hamas’ response to Trump’s Gaza peace plan win...,https://nypost.com/2025/10/04/world-news/hamas...,134
145,2025-10-04-03-10-45 +0000,nypost,"Hamas internal rift over hostages, weapons cou...",https://nypost.com/2025/10/03/world-news/hamas...,131
198,2025-10-03-22-22-09 +0000,nyt,Trump Calls on Israel to Stop Bombing Gaza Aft...,https://www.nytimes.com/2025/10/03/us/politics...,129
64,2025-10-04-15-14-32 +0000,wapo,Israel pauses Gaza City push after Hamas accep...,https://www.washingtonpost.com/world/2025/10/0...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
162,150,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...
49,53,2025-10-04-16-00-00 +0000,wsj,"To hear the top House Democrat tell it, beyond...",https://www.wsj.com/politics/policy/hakeem-jef...
136,47,2025-10-04-06-23-16 +0000,bbc,Sanae Takaichi set to become Japan's first fem...,https://www.bbc.com/news/articles/cx2pmy7m72lo...
44,45,2025-10-04-16-28-25 +0000,nypost,London police arrest dozens of protesters agai...,https://nypost.com/2025/10/04/world-news/londo...
217,40,2025-10-03-21-35-06 +0000,latimes,Supreme Court says again Trump may cancel temp...,https://www.latimes.com/politics/story/2025-10...
97,34,2025-10-04-12-00-00 +0000,wsj,Many government leaders have had the chance to...,https://www.wsj.com/economy/trade/south-korea-...
5,33,2025-10-04-19-37-12 +0000,nypost,New documentary examines hostage posters at ce...,https://nypost.com/2025/10/04/us-news/new-docu...
88,33,2025-10-04-12-37-30 +0000,cbc,Russian drone strike hits passenger train in U...,https://www.cbc.ca/news/world/russia-drone-ukr...
45,29,2025-10-04-16-23-45 +0000,nypost,Attorney General Pam Bondi says DOJ will appea...,https://nypost.com/2025/10/04/us-news/attorney...
245,29,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
